### NOTE 1
图卷积网络（Graph Convolutional Network, GCN）层在图神经网络中主要负责在图结构数据中进行特征的聚合与传播。具体而言，一个GCN层的数学意义可以用以下公式来表示：

$$H^{(l+1)} = \sigma\left(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)}\right)$$

其中：
- $H^{(l+1)}$ 表示第 $ l $ 层的节点特征矩阵，$H^{(0)}$ 就是输入的节点特征。
- $W^{(l)}$ 是第 $ l $ 层的权重矩阵，需要通过学习获得。
- $\hat{A}$ 是归一化后的图的邻接矩阵 $ A $ ，包括自连接（self-loop），一般表示为：$\hat{A} = \hat{D}^{-1/2} \hat{A} \hat{D}^{-1/2}$，其中 $\hat{A} = A + I$，$I$ 是单位矩阵，$\hat{D}$ 是 $\hat{A}$ 的度矩阵。
- $\sigma$ 是激活函数，如ReLU。

GCN层的核心思想是将每个节点的特征与其相邻节点的特征进行加权平均，从而实现信息在节点间的传播与聚合。这种方式能够有效地从图结构数据中提取局部信息及捕获全局上下文。通过层层GCN的叠加，网络能够学习到丰富的特征表示。

### NOTE 2
在图论中，给定一个图的邻接矩阵 $\hat{A}$，图的度矩阵 $\hat{D}$ 是一个对角矩阵，其对角元素 $\hat{D}_{ii}$ 表示图中对应节点 $i$ 的度（degree）。节点的度定义为与该节点相连的边的数量。

具体而言，$\hat{A}$ 是一个邻接矩阵，其中元素 $\hat{A}_{ij}$ 表示节点 $i$ 和节点 $j$ 之间是否存在边。当$\hat{A}$ 包含自连接时，即每个节点都被视为与自身相连，那么：

- $\hat{A}$ 的对角线元素通常为1，因为包括了自身的连接。
- $\hat{D}$ 的对角元素 $\hat{D}_{ii}$ 是节点 $i$ 的度 $\hat{D}_{ii} = \sum_j \hat{A}_{ij}$。

度矩阵 $\hat{D}$ 是图卷积网络中进行归一化的重要组成部分，它用于将 $\hat{A}$ 进行归一化处理，通过$\hat{D}^{-1/2} \hat{A} \hat{D}^{-1/2}$ 的操作来减轻节点度不均衡带来的影响，确保信息在传播和聚合的过程中得到更好的缩放控制。这种对称归一化有助于缓解图的不平衡性，提高特征传播的稳定性和有效性。

### NOTE 3
在图的邻接矩阵中，行和与列和的意义是相同的。如果一个图是无向图（即边没有方向），则邻接矩阵是对称的，因此行和和列和完全相同。

- 行和：某一行的元素的和代表与该行对应的节点相连的所有节点数，即该节点的度（degree）。
- 列和：某一列的元素的和也代表与该列对应的节点相连的所有节点数，因为对于无向图，节点的连接是互相的，因此度是相同的。

在无向图中，度矩阵的对角元素可以由邻接矩阵的任意一行或一列的和得到。

然而，在有向图中，邻接矩阵并不对称：

- 行和通常用于表示出度（out-degree），即从该节点出发的边的数量。
- 列和通常用于表示入度（in-degree），即指向该节点的边的数量。

在无向图中，出度和入度是相同的，因为每条边都是双向的，所以行和和列和的意义是相同的。而在有向图中，两者的意义不同。


在一个注意力机制的上下文中，例如具有形状为 `[batch_size, num_nodes, num_nodes, num_heads]` 的 `attn_matrix`，为其进行 `softmax` 操作的选择维度常常是 `dim=2`。这是因为：

1. **维度含义解析**:
   - **`batch_size`**: 表示批处理中的样本数量。
   - **第一个 `num_nodes`（`dim=1`）**: 通常表示当前输入作为查询节点的索引。
   - **第二个 `num_nodes`（`dim=2`）**: 表示可能作为键的节点索引，即查询节点与所有键节点之间的关系。
   - **`num_heads`**: 表示多头注意力中的不同头部。

2. **为什么选择 `dim=2`**:

   - 在注意力机制中，`softmax` 操作通常用于将某一维上的数值转换为概率分布。例如，给定一个查询节点，我们希望知道它在考虑所有的可能键节点时，如何分配注意力权重。
   
   - **`dim=2`（第二个 `num_nodes` 维度）**: 对这个维度执行 `softmax` 是为了计算每个查询节点与所有其他节点的注意力分数归一化。这种操作将得到一个概率分布，表示每个查询节点在其计算注意力时对其他节点的关注度。
   
   - 换句话说，对于每个特定的查询节点（确定后，沿着第一个 `num_nodes` 维度），我们沿着第二个 `num_nodes` 维度进行归一化，以处理它与所有键节点的注意力强度。

3. **多头注意力的考虑**:
   - 在多头注意力设置中，不同的头在并行工作，它们对相同的节点集计算独立注意力权重，因此维度 `dim=2` 适配每个头。

通过对第二个 `num_nodes` 维度（`dim=2`）执行 `softmax`，我们创建了一个关于每个查询节点与其所有键节点之间注意力关系的概率分布，这正是注意力机制要实现的核心目标。